In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from keras.applications.vgg16 import VGG16
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
os.chdir("/content/drive/MyDrive/DL_Hackathon/dataset")

In [ ]:
!ls

test  train


In [ ]:
real_path = "/content/drive/MyDrive/DL_Hackathon/dataset/train/training_real"
fake_path = "/content/drive/MyDrive/DL_Hackathon/dataset/train/training_fake"

In [ ]:
real_images = os.listdir(real_path)
fake_images = os.listdir(fake_path)

In [ ]:
real_images[0]

'1033.jpg'

In [ ]:
def load_img(path):
  image = cv2.imread(path)
  image = cv2.resize(image, (224, 224))
  return image[..., ::-1]

In [ ]:
dataset_path = "../dataset/train/"

In [ ]:
data_with_aug = ImageDataGenerator( rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip = True, vertical_flip = False, rescale = 1./255, validation_split = 0.1)
# data_with_aug = ImageDataGenerator()

In [ ]:
train = data_with_aug.flow_from_directory(dataset_path, target_size = (96, 96), batch_size = 32, subset = 'training')

Found 1539 images belonging to 2 classes.


In [ ]:
val = data_with_aug.flow_from_directory(dataset_path, target_size = (96, 96), batch_size = 32, subset = 'validation')

Found 170 images belonging to 2 classes.


In [ ]:
from keras.applications.vgg19 import VGG19
vggmodel = VGG19(weights='imagenet', include_top=True)

574726144/574710816 [==============================] - 3s 0us/step


In [ ]:
model = Sequential([vggmodel.layers[-5],
                    GlobalAveragePooling2D(),
                    Dense(512, activation = "relu"),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(128, activation = "relu"),
                    Dropout(0.1),
                    Dense(2, activation = "sigmoid")])

model.layers[0].trainable = False

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("vgg19_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')

hist = model.fit(train, steps_per_epoch= 1539//32, epochs= 100, validation_data= val, validation_steps=170//32, callbacks=[checkpoint,early])

Epoch 1/100
48/48 [==============================] - ETA: 0s - loss: 0.6993 - accuracy: 0.5514
Epoch 1: val_accuracy improved from -inf to 0.53750, saving model to vgg19_model.h5
48/48 [==============================] - 208s 4s/step - loss: 0.6993 - accuracy: 0.5514 - val_loss: 0.6924 - val_accuracy: 0.5375
Epoch 2/100
48/48 [==============================] - ETA: 0s - loss: 0.6939 - accuracy: 0.5554
Epoch 2: val_accuracy did not improve from 0.53750
48/48 [==============================] - 23s 476ms/step - loss: 0.6939 - accuracy: 0.5554 - val_loss: 0.6926 - val_accuracy: 0.5375
Epoch 3/100
48/48 [==============================] - ETA: 0s - loss: 0.6951 - accuracy: 0.5335
Epoch 3: val_accuracy improved from 0.53750 to 0.54375, saving model to vgg19_model.h5
48/48 [==============================] - 23s 476ms/step - loss: 0.6951 - accuracy: 0.5335 - val_loss: 0.6922 - val_accuracy: 0.5437
Epoch 4/100
48/48 [==============================] - ETA: 0s - loss: 0.7008 - accuracy: 0.5355
Epoc

In [ ]:
train.class_indices

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1, figsize = (9, 7))
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs. val_loss')
plt.grid(True)
plt.legend(['train', 'val'])
plt.show()
# plt.style.use(['classic'])



In [ ]:
plt.figure(figsize = (9, 7))
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('train_accuracy vs. val_accuracy')
plt.grid(True)
plt.legend(['train','val'], loc = 4)
plt.show()
# plt.style.use(['classic'])

In [ ]:
test_path = "/content/drive/MyDrive/DL_Hackathon/dataset/test/test_images"

In [ ]:
test_images = os.listdir(test_path)

In [ ]:
plt.figure(figsize = (12, 12))

for i in range(16):
  plt.subplot(4, 4, i + 1)
  plt.tight_layout()
  plt.imshow(load_img(test_path + test_images[i]), cmap = 'gray')
  plt.suptitle('Test images', fontsize = 16, fontweight = 'bold', y = 1.02)

plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
test = test_datagen.flow_from_directory(directory = "/content/drive/MyDrive/DL_Hackathon/dataset/test/test_images", class_mode = None, target_size = (96, 96), batch_size = 1, shuffle = False)

In [ ]:
predictions = model.predict(test, steps = 332)

In [ ]:
predictions[:20]

In [ ]:
predicted_class = predictions.round()

In [ ]:
predicted_class[:5]

In [ ]:
# predicted_class = predicted_class.reshape(1, -1)
predicted_class = predicted_class.tolist()

In [ ]:
predicted_class = np.argmax(predictions, axis = 1)

In [ ]:
file_id = [x.replace('test_images/', '').replace('.jpg', '') for x in test.filenames]

In [ ]:
output = pd.DataFrame({'file_id': file_id, 'label': predicted_class})

In [ ]:
output.head()

In [ ]:
output.to_csv("test_data.csv", index = False)